# Homework Assignment 7 - Evan Callaghan

### 1. Which of the following is/are TRUE about gradient boosting trees?

#### F. In gradient boosting trees, the decision trees are dependent of each other and it is a method for improving the performance by aggregating the results of several decision trees.

### 2. Suppose you are given the following scenario for training and validation for gradient boosting trees. Which scenario would you choose?

#### I would choose Scenario 2 with a max depth of four. Scenario 3, 4, and 5 are all examples of over-fitting the data because they have very low training errors and high validation errors. Left with Scenario 1 and 2, Scenario 2 is the clear choice because it has less validation error for only a slight increase in model complexity when increasing the depth from two to four.

### 3. In this course have covered two boosting frameworks. What is the main difference between AdaBoost and Gradient Boosting?

#### The main difference between the AdaBoost and Gradient Boosting frameworks is how they work to minimize the total error. AdaBoost handles miscalssifications by adjusting observation weights so the model is forced to learn how to correctly classify or predict on those observations in order to minimize the overall cost. On the other hand, Gradient Boosting finds the observations with large residuals in the previous iteration and shifts the focus of the model to minimize each individual loss.

### 4. Does the gradient boosting classifier model always outperforms the random forest model?

#### No, the gradient boosting classifier model does not always outperform the random forest model. Although it may be thought of as the better model, there is no mathematical proof to ensure that it will always outperform a different type of model.

### 5. Which of the following ensemble learning model does not include a learning rate parameter?

#### A. The Random Forest learning model does not include a learning rate parameter.

In [3]:
## 6. a) Using the pandas library to read the csv data file and create a data-frame called heart
## and removing the observations with missing values

import boto3
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, accuracy_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

heart = pd.read_csv(file_content_stream)

## Removing observation with missing values
heart = heart.dropna()

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
## b) Using age, totChol, sysBP,BMI, heartRate, and glucose as the predictor variables and TenYearCHD as the target 
## variable to do the following:

## Defining the input and target variables:
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Defining empty lists to store results
md1_recall = []
md1_accuracy = []
md2_recall = []
md2_accuracy = []
md3_recall = []
md3_accuracy = []


## Repeating 100 times:

for i in tqdm(range(0, 100)):
    
    ## Splitting the data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    
    ## Model 1
    ## ---------------------------------------
    
    ## Building a random forest model (with 500 trees and max tree depth equal 3) 
    md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    
    ## Predicting on the test set
    md1_preds = md1.predict_proba(X_test)[:, 1]
    
    ## Using 10% as cutoff value
    md1_preds = np.where(md1_preds < 0.1, 0, 1)
    
    ## Reporting the accuracy and recall of the model
    md1_recall.append(recall_score(Y_test, md1_preds))
    md1_accuracy.append(accuracy_score(Y_test, md1_preds))
    
    
    ## Model 2
    ## ---------------------------------------
    
    ## Building an AdaBoost model (with 500 trees, max tree depth equal 3, and learning rate equal to 0.01)
    md2 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, 
                             learning_rate = 0.01).fit(X_train, Y_train)
    
    ## Predicting on the test set
    md2_preds = md2.predict_proba(X_test)[:, 1]
    
    ## Using 10% as cutoff value
    md2_preds = np.where(md2_preds < 0.1, 0, 1)
    
    ## Reporting the accuracy and recall of the model
    md2_recall.append(recall_score(Y_test, md2_preds))
    md2_accuracy.append(accuracy_score(Y_test, md2_preds))
    
    
    ## Model 3
    ## ---------------------------------------
    
    ## Building a gradient boosting model (with 500 trees, max tree depth equal 3, and learning rate equal to 0.01) 
    md3 = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)
    
    ## Predicting on the test set
    md3_preds = md3.predict_proba(X_test)[:, 1]
    
    ## Using 10% as cutoff value
    md3_preds = np.where(md3_preds < 0.1, 0, 1)
    
    ## Reporting the accuracy and recall of the model
    md3_recall.append(recall_score(Y_test, md3_preds))
    md3_accuracy.append(accuracy_score(Y_test, md3_preds))
    
    
## Reporting the average accuracy and average recall for each model
print('Model 1: Accuracy -', np.mean(md1_accuracy), '   Recall -', np.mean(md1_recall))
print('Model 2: Accuracy -', np.mean(md2_accuracy), '   Recall -', np.mean(md2_recall)) 
print('Model 3: Accuracy -', np.mean(md3_accuracy), '   Recall -', np.mean(md3_recall))

100%|██████████| 100/100 [08:18<00:00,  4.98s/it]

Model 1: Accuracy - 0.4551229508196722    Recall - 0.8519642857142857
Model 2: Accuracy - 0.156775956284153    Recall - 0.9925000000000002
Model 3: Accuracy - 0.5094808743169398    Recall - 0.8035714285714285


In [ ]:
## c) Assuming that the ideal model needs to have a minimum accuracy and recall equal to 80%, 
## the best model from part (b) does not meet this requirement. I would try to tune the number 
## of trees, the learning rate, and the cut-off value in order for the model to reach the 
## minimum requirements. 